In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

### Load the prediction model and test

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
bs = 48

In [5]:
data_clas = load_data(path, 'data_clas.pkl', bs=bs) # this loads the vocabulary into a data classifier databunch

In [6]:
# here we create a classifier using the vocab of the language model (the databunch) and the base model AWD_LSTM
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16() 

In [8]:
# this loads the encoder from the language model - the part that converts text to vector form
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (25000 items)
x: TextList
xxbos i find i enjoy this show , but the format needs some work . xxmaj first off , the good attributes . i like how this show will take us through the day - to - day life of an addict because the producers have a knack at getting the addict to show us how bad they 've allowed their lives to become . xxmaj this is followed by an intervention which is then followed by an outcome . xxmaj intervention does n't candy - coat things and sometimes the outcome ( often short term due to the constraints of time between filming and airing ) is a negative outcome . xxmaj this makes the positive outcomes all the better . 
 
  xxmaj another thing i like about the show is the quality of the camera work . xxmaj given the reality that these cameramen have to squeeze anywhere and do n't have the benefit of re shooting scenes the photography is surprisingly good and stable . xxmaj it 's actually superior to scripted shows like

In [9]:
learn.load('third') # load the pre-trained weights from the trained model, the classifier logic per se

RNNLearner(data=TextClasDataBunch;

Train: LabelList (25000 items)
x: TextList
xxbos i find i enjoy this show , but the format needs some work . xxmaj first off , the good attributes . i like how this show will take us through the day - to - day life of an addict because the producers have a knack at getting the addict to show us how bad they 've allowed their lives to become . xxmaj this is followed by an intervention which is then followed by an outcome . xxmaj intervention does n't candy - coat things and sometimes the outcome ( often short term due to the constraints of time between filming and airing ) is a negative outcome . xxmaj this makes the positive outcomes all the better . 
 
  xxmaj another thing i like about the show is the quality of the camera work . xxmaj given the reality that these cameramen have to squeeze anywhere and do n't have the benefit of re shooting scenes the photography is surprisingly good and stable . xxmaj it 's actually superior to scripted shows like

In [10]:
learn.predict('mindhunter is awesome')

(Category pos, tensor(1), tensor([0.3056, 0.6944]))

In [12]:
#export the whole model
learn.export('export.pkl')

### Another load approach

In [13]:
learn


RNNLearner(data=TextClasDataBunch;

Train: LabelList (25000 items)
x: TextList
xxbos i find i enjoy this show , but the format needs some work . xxmaj first off , the good attributes . i like how this show will take us through the day - to - day life of an addict because the producers have a knack at getting the addict to show us how bad they 've allowed their lives to become . xxmaj this is followed by an intervention which is then followed by an outcome . xxmaj intervention does n't candy - coat things and sometimes the outcome ( often short term due to the constraints of time between filming and airing ) is a negative outcome . xxmaj this makes the positive outcomes all the better . 
 
  xxmaj another thing i like about the show is the quality of the camera work . xxmaj given the reality that these cameramen have to squeeze anywhere and do n't have the benefit of re shooting scenes the photography is surprisingly good and stable . xxmaj it 's actually superior to scripted shows like

### Process some tweets

In [49]:
import pathlib
import json
import numpy as np
import pandas as pd
from IPython.display import display

In [11]:
tweet_path = pathlib.Path('/home/jupyter/my_projects/tv_tweets/')

In [13]:
programs = [{'name':'mindhunter','file':'mindhunter_tweets.json'},
           {'name':'designated survivor','file':'des_sur_tweets.json'},
           {'name':'the i-land','file':'i_land_tweets.json'},
           {'name':'another life','file':'al_tweets.json'},]

In [52]:
tweet_file = tweet_path / 'al_tweets.json'

In [53]:
with open(tweet_file,'r') as f:
    tweets = json.load(f)

In [54]:
len(tweets)

100

In [55]:
for t in tweets[0:5]:
    print(t)
    print(learn.predict(t))

recently watched Another Life on @netflix and i really liked it. i dont get all the hate. Sure its not Star Wars, b… https://t.co/7y6bWiz8pk
(Category pos, tensor(1), tensor([0.0227, 0.9773]))
@jasonrdavis I’m afraid for a channel heavy in sci fi, most of the Netflix shows are shite. ST Discovery rides a co… https://t.co/DzxI71PQXx
(Category neg, tensor(0), tensor([0.6372, 0.3628]))
RT @RedFiddler: I just tried to watch Another Life on Netflix. In the first episode, the female protagonist - the captain of an intergalact…
(Category neg, tensor(0), tensor([0.7742, 0.2258]))
#Netflix’s IN THE TALL GRASS Trailer Brings Another #StephenKing Tale To Life

#IntheTallGrass https://t.co/eRLgKyyWyU
(Category pos, tensor(1), tensor([0.2441, 0.7559]))
I just tried to watch Another Life on Netflix. In the first episode, the female protagonist - the captain of an int… https://t.co/D9D7KTNMpE
(Category pos, tensor(1), tensor([0.4383, 0.5617]))


In [35]:
def get_scores(tweets):
    
    cats = []
    scores = []
    
    for tweet in tweets:
        cat, ten, score = learn.predict(tweet)
    
        cats.append(cat.__str__())
        scores.append(score[ten].item())
        
    return cats, scores
    
    

In [36]:
cats, scores = get_scores(tweets)

In [43]:
df = pd.DataFrame(data={'Categories':cats,'Scores':scores})
df.head()

,Categories,Scores
0,neg,0.696478
1,neg,0.828244
2,pos,0.679582
3,pos,0.542215
4,neg,0.507333


In [47]:
df.groupby('Categories').agg({'Categories':len,'Scores':np.mean})

,Categories,Scores
Categories,,
neg,39,0.674914
pos,61,0.721752


In [50]:
for program in programs:
    print(program['name'])
    
    #load tweets
    tweet_file = tweet_path / program['file']
    
    with open(tweet_file,'r') as f:
        tweets = json.load(f)
    
    cats, scores = get_scores(tweets)
    df = pd.DataFrame(data={'Categories':cats,'Scores':scores})
    display(df.groupby('Categories').agg({'Categories':len,'Scores':np.mean}))
    

mindhunter


,Categories,Scores
Categories,,
neg,39,0.674914
pos,61,0.721752


designated survivor


,Categories,Scores
Categories,,
neg,35,0.714103
pos,65,0.755773


the i-land


,Categories,Scores
Categories,,
neg,59,0.772427
pos,41,0.763920


another life


,Categories,Scores
Categories,,
neg,31,0.761778
pos,69,0.783529
